https://www.tutorialspoint.com/python_blockchain/index.htm

In [151]:
import hashlib
import random
import string
import json
import binascii
import numpy as np
import pandas as pd
import pylab as pl
import logging
import datetime
import collections


In [152]:
import Crypto
import Crypto.Random
from Crypto.Hash import SHA
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5

In [153]:
class Client:
    def __init__(self):
        random = Crypto.Random.new().read
        self._private_key = RSA.generate(1024, random)
        self._public_key = self._private_key.publickey()
        self._signer = PKCS1_v1_5.new(self._private_key)

    @property
    def identity(self):
        return binascii.hexlify(self._public_key.exportKey(format='DER')).decode('ascii')

In [155]:
c = Client()
c.identity

'30819f300d06092a864886f70d010101050003818d0030818902818100cce1b7df40ee619403d934a1b99fc7eabe8294d5be101bd8c8376cd1b2ed2cbb5a6ab68be53a4dc9d3c12ba4bc3238495428436242f5043cb7ca2d3729dab0a386952fdaf070518f6aead3abadb2d02c13acff92b06bfb6047fc843345eaaaa7e448638ddf0405bac79098674565ee2df7fedab2e529c5d5f71722a0133602bb0203010001'

In [160]:
print(binascii.hexlify(c._private_key.exportKey(format='DER')).decode('ascii'))
# type(binascii.hexlify(c._private_key.exportKey(format='DER')))
# type(binascii.hexlify(c._private_key.exportKey(format='DER')).decode('ascii'))

3082025d02010002818100cce1b7df40ee619403d934a1b99fc7eabe8294d5be101bd8c8376cd1b2ed2cbb5a6ab68be53a4dc9d3c12ba4bc3238495428436242f5043cb7ca2d3729dab0a386952fdaf070518f6aead3abadb2d02c13acff92b06bfb6047fc843345eaaaa7e448638ddf0405bac79098674565ee2df7fedab2e529c5d5f71722a0133602bb0203010001028180200f25e582548ae24d63dc8f7deb59ae0ad7861098280f2371d8f26731d7590b8c0f6cb7214c6945aea3e87f638f7c2d4544f39e5999db6ae0db7b0dc9b7d5b1c9340024b9708679a2243be90acad72312dfd8a2fff9bb412af0c0bb638dd1d085c015e66a290d1d2575157f9c00305eba2f21346cfda8b83b1b74fa6277a4a1024100dd905a350477c7ee0e9e44a39f7d4eddaf7c2e4c6b32f00b47de39551ad77e1b7e6e1c63efd042df76d2357de59121b2050a7d383aedca5d64b07478874293f1024100ecb99c7055ef3413a0b3c199b97c6d8ff1ef6024e106bcb985f96798e3399562e5004e573dd34bad24cd63f7bda61b9ecf22b668f6056b62822ec29419fdfd6b024100c067aaee7f82c54c25c5ab35ac3ad5520e2278d24405680f63c9c75ff07f5233ee071100783112e19bfdd0161455843d63c3481a9c92033a374644dea6175b01024100b98ab5c21aa8d5117948cb3a98f69ccd511c852802

In [120]:
class Transaction:
    def __init__(self, sender, recipient, value):
        self.sender = sender
        self.recipient = recipient
        self.value = value
        self.time = datetime.datetime.now()
        
    def to_dict(self):
        if self.sender == "Genesis":
            identity = "Genesis"
        else:
            identity = self.sender.identity

        return collections.OrderedDict({'sender': identity, 'recipient': self.recipient, 
                                        'value': self.value, 'time' : self.time})
    
    def sign_transaction(self):
        private_key = self.sender._private_key
        signer = PKCS1_v1_5.new(private_key)
        h = SHA.new(str(self.to_dict()).encode('utf8'))
        return binascii.hexlify(signer.sign(h)).decode('ascii')


In [121]:
c1 = Client()
c2 = Client()

In [122]:
t = Transaction(c1, c2.identity, 5.0)

In [123]:
signature = t.sign_transaction()
signature

'5aa1ea9ff81c2d012f225023037e5b151a69142acb2838c231121c0ba1328e4448bc158e1233022a99225cc1409be49d1f2ef65d01a24309fb9be870850bef35b4df7c90db21c4375519b71dda0ebe6fb0deccaf3c28279b950c26dd22f32a05467518e6bce91adc9fa4290eee7fd6bad0fb47d3df74a9f644c73c7b2663a733'

In [124]:
def display_transaction(transaction):
    #for transaction in transactions:
    dict = transaction.to_dict()
    print("sender: " + dict['sender'])
    print('-----')
    print("recipient: " + dict['recipient'])
    print('-----')
    print("value: " + str(dict['value']))
    print('-----')
    print("time: " + str(dict['time']))
    print('-----')

In [125]:
display_transaction(t)

sender: 30819f300d06092a864886f70d010101050003818d003081890281810098b604bf7ba093b68fd35872de6edc52efe75cdfbb2005068d7d8ef65af73d7eafbf2ce91062e96818b8ed2706dd60bd095195b058b0652b779a34d58625d82114734b2a391c54eea52f603934b972e85dea212b5a80ce123703c57ab6dcfef5f7a65ddc9bfec4034d459fdf31fc93ab697af078799df90e267b363b8a6cac1b0203010001
-----
recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100c783c372529dcefa4605d35aaf3c6ecd1f06d46a2e331397fe34d1cc12c73ca3be06808435d383103b25fd67839c055c1ac426fc3260448ac3fc414c5559e659f403654ac9d77eb22b5e72597ff9ca3135db9bc6e92d5c05bdb522056ef0722e5a35c8888b33581dd5d0f7ae6ea2e2185383c97e3d86a122b2870c08f5ad450b0203010001
-----
value: 5.0
-----
time: 2022-03-28 22:15:15.872074
-----


In [126]:
transactions = []

In [127]:
c3 = Client()
c4 = Client()

In [128]:
t1 = Transaction(c1, c2.identity, 15.0)

In [129]:
t1.sign_transaction()
transactions.append(t1)

In [130]:
t2 = Transaction(c3, c4.identity, 6.0)
t2.sign_transaction()
transactions.append(t2)

t3 = Transaction(c1, c4.identity, 2.0)
t3.sign_transaction()
transactions.append(t3)

t4 = Transaction(c4, c2.identity, 12.0)
t4.sign_transaction()
transactions.append(t4)

t5 = Transaction(c3, c4.identity, 2.0)
t5.sign_transaction()
transactions.append(t5)

t6 = Transaction(c3, c1.identity, 22.0)
t6.sign_transaction()
transactions.append(t6)

t7 = Transaction(c2, c4.identity, 125.0)
t7.sign_transaction()
transactions.append(t7)

t8 = Transaction(c4, c3.identity, 42.0)
t8.sign_transaction()
transactions.append(t8)

t9 = Transaction(c3, c1.identity, 2.0)
t9.sign_transaction()
transactions.append(t9)

t10 = Transaction(c2, c1.identity, 5.0)
t10.sign_transaction()
transactions.append(t10)

In [131]:
for transaction in transactions:
       display_transaction(transaction)
       print('--------------')

sender: 30819f300d06092a864886f70d010101050003818d003081890281810098b604bf7ba093b68fd35872de6edc52efe75cdfbb2005068d7d8ef65af73d7eafbf2ce91062e96818b8ed2706dd60bd095195b058b0652b779a34d58625d82114734b2a391c54eea52f603934b972e85dea212b5a80ce123703c57ab6dcfef5f7a65ddc9bfec4034d459fdf31fc93ab697af078799df90e267b363b8a6cac1b0203010001
-----
recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100c783c372529dcefa4605d35aaf3c6ecd1f06d46a2e331397fe34d1cc12c73ca3be06808435d383103b25fd67839c055c1ac426fc3260448ac3fc414c5559e659f403654ac9d77eb22b5e72597ff9ca3135db9bc6e92d5c05bdb522056ef0722e5a35c8888b33581dd5d0f7ae6ea2e2185383c97e3d86a122b2870c08f5ad450b0203010001
-----
value: 15.0
-----
time: 2022-03-28 22:15:16.430052
-----
--------------
sender: 30819f300d06092a864886f70d010101050003818d0030818902818100b9ec55650f19c77f14a26c37fb8019322be77cac4c9db6b7f3cc27affe66bf33b606fc5f0bad928731f64d4d6200f66365d04e39e0af5499e37e5997f57025fe43dac54f14aaaccdb2cf1a02dfce060cce4a2ba32826a86c91881

https://www.tutorialspoint.com/python_blockchain/python_blockchain_block_class.htm

In [132]:
class Block:
    def __init__(self):
        self.verified_transactions = []
        self.previous_block_hash = ""
        self.Nonce = ""
        

In [133]:
last_block_hash = ""

In [134]:
t0 = Transaction("Genesis", c1.identity, 500.0)

In [135]:
block0 = Block()

In [136]:
block0.previous_block_hash = None
Nonce = None

In [137]:
block0.verified_transactions.append(t0)

In [138]:
digest = hash(block0)
last_block_hash = digest

In [139]:
TPCoins = []

In [140]:
def dump_blockchain(self):
    print("Number of blocks in the chain: " + str(len(self)))
    
    for x in range(len(TPCoins)):
        block_temp = TPCoins[x]
        print("block # " + str(x))
        for transaction in block_temp.verified_transactions:
            display_transaction(transaction)
            print('--------------')
        print('=====================================')

In [141]:
TPCoins.append(block0)

In [142]:
dump_blockchain(TPCoins)

Number of blocks in the chain: 1
block # 0
sender: Genesis
-----
recipient: 30819f300d06092a864886f70d010101050003818d003081890281810098b604bf7ba093b68fd35872de6edc52efe75cdfbb2005068d7d8ef65af73d7eafbf2ce91062e96818b8ed2706dd60bd095195b058b0652b779a34d58625d82114734b2a391c54eea52f603934b972e85dea212b5a80ce123703c57ab6dcfef5f7a65ddc9bfec4034d459fdf31fc93ab697af078799df90e267b363b8a6cac1b0203010001
-----
value: 500.0
-----
time: 2022-03-28 22:15:16.553412
-----
--------------


https://www.tutorialspoint.com/python_blockchain/python_blockchain_creating_miners.htm

In [143]:
def sha256(message):
    return hashlib.sha256(message.encode('ascii')).hexdigest()

In [144]:
def mine(message, difficulty=1):
    assert difficulty >= 1
    prefix = '1' * difficulty
    for i in range(10000):
        digest = sha256(str(hash(message)) + str(i))
        if digest.startswith(prefix):
            print("after " + str(i) + " iterations found nonce: "+ digest)
            return digest


In [145]:
mine("this is a really long message", 3)

after 5824 iterations found nonce: 111926b32057fda34eec960f99e5351da836087f284972af8bea906e6906324b


'111926b32057fda34eec960f99e5351da836087f284972af8bea906e6906324b'

In [146]:
last_transaction_index = 0

In [147]:
block = Block()
for i in range(3):
    temp_transaction = transactions[last_transaction_index]
    # validate transaction
    # if valid
    block.verified_transactions.append(temp_transaction)
    last_transaction_index += 1

block.previous_block_hash = last_block_hash
block.Nonce = mine(block, 2)
digest = hash(block)
TPCoins.append(block)
last_block_hash = digest

after 35 iterations found nonce: 117eced78be699210194d74e7303bf0235b9cb646710300f993b04bf505a5688


In [148]:
TPCoins

[<__main__.Block at 0x1117e5790>, <__main__.Block at 0x1117efd60>]

In [149]:
# Miner 2 adds a block
block = Block()

for i in range(3):
    temp_transaction = transactions[last_transaction_index]
    # validate transaction
    # if valid
    block.verified_transactions.append(temp_transaction)
    last_transaction_index += 1
block.previous_block_hash = last_block_hash
block.Nonce = mine(block, 2)
digest = hash(block)
TPCoins.append(block)
last_block_hash = digest


# Miner 3 adds a block
block = Block()

for i in range(3):
    temp_transaction = transactions[last_transaction_index]
    #display_transaction(temp_transaction)
    # validate transaction
    # if valid
    block.verified_transactions.append(temp_transaction)
    last_transaction_index += 1

block.previous_block_hash = last_block_hash
block.Nonce = mine(block, 2)
digest = hash(block)

TPCoins.append(block)
last_block_hash = digest


after 118 iterations found nonce: 1191d3ef239101a33aae48912d72ca728a249587d56fd36e2ed257531d45adca
after 1453 iterations found nonce: 113eb568bada80a1bd7e355ac55c225b30dfad1b442ad6d73b3c3c013479d9df


In [150]:
dump_blockchain(TPCoins)

Number of blocks in the chain: 4
block # 0
sender: Genesis
-----
recipient: 30819f300d06092a864886f70d010101050003818d003081890281810098b604bf7ba093b68fd35872de6edc52efe75cdfbb2005068d7d8ef65af73d7eafbf2ce91062e96818b8ed2706dd60bd095195b058b0652b779a34d58625d82114734b2a391c54eea52f603934b972e85dea212b5a80ce123703c57ab6dcfef5f7a65ddc9bfec4034d459fdf31fc93ab697af078799df90e267b363b8a6cac1b0203010001
-----
value: 500.0
-----
time: 2022-03-28 22:15:16.553412
-----
--------------
block # 1
sender: 30819f300d06092a864886f70d010101050003818d003081890281810098b604bf7ba093b68fd35872de6edc52efe75cdfbb2005068d7d8ef65af73d7eafbf2ce91062e96818b8ed2706dd60bd095195b058b0652b779a34d58625d82114734b2a391c54eea52f603934b972e85dea212b5a80ce123703c57ab6dcfef5f7a65ddc9bfec4034d459fdf31fc93ab697af078799df90e267b363b8a6cac1b0203010001
-----
recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100c783c372529dcefa4605d35aaf3c6ecd1f06d46a2e331397fe34d1cc12c73ca3be06808435d383103b25fd67839c055c1ac426

https://www.tutorialspoint.com/python_blockchain/python_blockchain_scope_and_conclusion.htm